In [ ]:
import numpy as np
import scipy
import torch
from typing_extensions import final
import torch.optim as optim
import statistics  # For mean and variance calculations
import os
from numpy import genfromtxt
import pickle
import random
# from matplotlib import pyplot as plt
# initialize variables
mole_name = "OHH"
n_orb = 12
na_true = 4
nb_true = 4
# previous code naming N
N = n_orb
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

In [ ]:
!pip install pyscf
from google.colab import drive
drive.mount('/content/drive')
####Check if running in Google Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 MB 41.1 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
def scipy_to_torch_sparse(scipy_csr_matrix):
    """
    Convert a SciPy sparse CSR matrix (with float64 data) to a PyTorch sparse CSR tensor (with complex64 data),
    minimizing extra memory usage.
    Parameters
    ----------
    scipy_csr_matrix : scipy.sparse.csr_matrix
        The input SciPy CSR matrix with float64 data.

    Returns
    -------
    torch.sparse_csr_tensor
        The converted PyTorch sparse CSR tensor with complex64 data.
    """

    if not isinstance(scipy_csr_matrix, scipy.sparse.csr_matrix):
        raise TypeError("Input must be a SciPy CSR matrix.")

    # Extract pointer/indices/data
    row_ptr = scipy_csr_matrix.indptr   # shape: (n_rows + 1,)
    col_idx = scipy_csr_matrix.indices  # shape: (nnz,)
    data = scipy_csr_matrix.data        # shape: (nnz,)
    shape = scipy_csr_matrix.shape

    # Delete the SciPy object to free overhead memory
    del scipy_csr_matrix

    # Create PyTorch tensors by sharing memory with NumPy
    row_ptr_tensor = torch.from_numpy(row_ptr)
    col_idx_tensor = torch.from_numpy(col_idx)
    data_tensor = torch.from_numpy(data)

    # Delete the NumPy arrays once we have PyTorch references
    del row_ptr, col_idx, data

    # Ensure row_ptr/col_idx are torch.long (int64)
    if row_ptr_tensor.dtype != torch.long:
        row_ptr_tensor = row_ptr_tensor.long()
    if col_idx_tensor.dtype != torch.long:
        col_idx_tensor = col_idx_tensor.long()

    # Convert float64 -> complex128 (this is our one unavoidable copy)
    data_tensor = data_tensor.to(torch.complex128)

    # Create the final sparse CSR tensor
    torch_csr_tensor = torch.sparse_csr_tensor(
        crow_indices=row_ptr_tensor,
        col_indices=col_idx_tensor,
        values=data_tensor,
        size=shape
    )
    return torch_csr_tensor

In [ ]:
bond_length = 1
basis = ['sto-6g','6-31g']
basis_choice= 1
Mo_AO_basis = ['MO','local_MO']
Mo_AO_basis_choice = 1
string_name = f'{mole_name}_bond_{bond_length}_Ham_{basis[basis_choice]}_{Mo_AO_basis[Mo_AO_basis_choice]}.npz'
string_name_gs = f'{mole_name}_bond_{bond_length}_true_gs_{basis[basis_choice]}_{Mo_AO_basis[Mo_AO_basis_choice]}.csv'

if IN_COLAB:
    file_path = f'/content/drive/My Drive/Quantum_chemistry/data/Water/states/{string_name}'
    file_path_gs = f'/content/drive/My Drive/Quantum_chemistry/data/Water/states/{string_name_gs}'
else:
    file_path = string_name
    file_path_gs = string_name_gs

h_sparse = scipy.sparse.load_npz(file_path)
h_sparse = scipy_to_torch_sparse(h_sparse)
h_sparse = h_sparse.to(device)

# Load the eigenvectors
true_gs = np.loadtxt(file_path_gs, delimiter=',')

# move to gpu
true_gs = torch.tensor(true_gs, dtype=torch.complex128).to(device)


/tmp/ipython-input-3-2727545122.py:46: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  torch_csr_tensor = torch.sparse_csr_tensor(


In [ ]:
# true_gs = torch.tensor(true_gs, dtype=torch.float64).to(device)

# round_gs = torch.round(true_gs,decimals=5)
# round_gs = torch.tensor(round_gs, dtype=torch.complex128).to(device)
# calculate_energy(h_sparse,round_gs)

In [ ]:
# bond_length = 1
# basis = ['sto-6g','6-31g']
# basis_choice= 1
# Mo_AO_basis = ['MO','local_MO']
# Mo_AO_basis_choice = 1
# string_name = f'{mole_name}_bond_{bond_length}_Ham_{basis[basis_choice]}_{Mo_AO_basis[Mo_AO_basis_choice]}.npz'
# string_name_gs = f'{mole_name}_bond_{bond_length}_true_gs_{basis[basis_choice]}_{Mo_AO_basis[Mo_AO_basis_choice]}.csv'
# if IN_COLAB:
#     file_path = f'/content/drive/My Drive/Quantum_chemistry/data/Water/states/{string_name}'
#     file_path_gs = f'/content/drive/My Drive/Quantum_chemistry/data/Water/states/{string_name_gs}'
# else:
#     file_path = string_name
#     file_path_gs = string_name_gs
# true_gs_1 = np.loadtxt(file_path_gs, delimiter=',')
# true_gs_1 = torch.tensor(true_gs_1, dtype=torch.complex128).to(device)


In [ ]:
def calculate_energy(input_matrix, psi): # input_matrix is sparse
    psi_dagger = torch.conj(psi)  # Conjugate of the state vector
    # Sparse matrix-vector multiplication
    intermediate = torch.sparse.mm(input_matrix, psi.unsqueeze(1))  # shape (n, 1) # support coo or csr matrix
    # Dot product to get the energy
    energy = torch.dot(psi_dagger, intermediate.squeeze())  # shape (n,)
    return energy.real

In [ ]:
torch.set_printoptions(precision=10)
true_gs_energy = calculate_energy(h_sparse,true_gs)
print(true_gs_energy)

tensor(-19.4132131926, device='cuda:0', dtype=torch.float64)


In [ ]:
# gs energy list for OHH:
# r =1, -23.9984226491986
# r =1.01, -23.7040
# r= 1.25, -22.3022
# r= 1.5 , -21.3267
# r =1.75,  -20.6262
# r =2, -20.1095
# r =2.05, -20.0227946472036
# r =2.1,-19.940666061819677
# r =2.15, -19.8628087583110
# r =2.2,  -19.7888961801287
# r =2.25, -19.7186
# r = 2.5,-19.4132

In [ ]:
# start with double occ state, one higher than HF
from pyscf.fci import cistring
alpha_list = list(cistring.gen_strings4orblist(range(n_orb), na_true))
beta_list  = list(cistring.gen_strings4orblist(range(n_orb), nb_true))

# Build a list of (alpha, beta) pairs in EXACT the same order as PySCF:
basis_pair = []
for a in alpha_list:
    for b in beta_list:
        basis_pair.append((a, b))

# Find the second occurrence of a = b
matching_indices = [i for i, (a, b) in enumerate(basis_pair) if a == b]

state_to_evolve = torch.zeros(h_sparse.shape[0], dtype=torch.complex128)

################ line to search initial state
# basis_pair.index((np.int64(4368), np.int64(4368)))
################
state_to_evolve[matching_indices[0]] = 1
state_to_evolve = state_to_evolve/torch.norm(state_to_evolve)
print(torch.norm(state_to_evolve))
# Move to GPU
state_to_evolve = state_to_evolve.to(device)


tensor(1., dtype=torch.float64)


In [ ]:
def calculate_fidelity(psi, phi):
    """
    Calculate the fidelity between two state vectors.

    Parameters:
    psi (torch.Tensor): The first state vector in PyTorch format, dtype=torch.complex128.
    phi (torch.Tensor): The second state vector in PyTorch format, dtype=torch.complex128.

    Returns:
    float: The calculated fidelity.
    """
    fidelity = torch.abs(torch.vdot(psi, phi))**2
    return fidelity.item()

In [ ]:
print(calculate_fidelity(state_to_evolve,true_gs))

7.646817425047819e-07


In [ ]:
# # check the energy of the state to evolve
print(calculate_energy(h_sparse,state_to_evolve))

tensor(-13.1049177341, device='cuda:0', dtype=torch.float64)


In [ ]:
# U part, read data
string_name = f'number_op_list_norb={N}_na_{na_true}_nb_{nb_true}.txt'
if IN_COLAB:
    file_path = f'/content/drive/My Drive/Quantum_chemistry/hardware_effient/{string_name}'
else:
    file_path = string_name

number_op_list = torch.tensor(np.loadtxt(file_path))
# move to gpu
number_op_list = number_op_list.to(device)

In [ ]:
def UCC_U_part_torch(input_state, para_index, input_theta):
    """
    Apply the UCC U part operation on the input state.

    Parameters:
    input_state (torch.Tensor): The input state tensor in PyTorch format, dtype=torch.complex128.
    para_index (int): The index to select the corresponding number operator from number_op_list.
    input_theta (float): The input theta parameter for the phase factor.

    Returns:
    torch.Tensor: The output state after applying the phase factors.
    """
    occ_list = number_op_list[para_index]

    # Compute the phase factors
    phase_factor_list = torch.exp(1j * input_theta * occ_list)

    # Apply the phase factors to the input state
    output_state = phase_factor_list * input_state

    return output_state

In [ ]:
# UCC_U_part_torch(state_to_evolve,2,torch.tensor([0.3],dtype=torch.float64).to(device ))

In [ ]:
# Specify the file path to save the dictionary
string_name = f'torch_hopping_cose_dict_norb={N}_na_{na_true}_nb_{nb_true}.pkl'

# Load the dictionary from the file
if IN_COLAB:
    file_path = f'/content/drive/My Drive/Quantum_chemistry/hardware_effient/{string_name}'
else:
    file_path = string_name

hopping_cose_dict = pickle.load(open(file_path, 'rb'))

# Convert numpy arrays to torch tensors
hopping_cose_dict = {key: torch.tensor(value, device=device) for key, value in hopping_cose_dict.items()}
###############################################################################################################################################################

string_name = f'torch_sparse_matrices_dict_norb={N}_na_{na_true}_nb_{nb_true}.pkl'
if IN_COLAB:
    file_path = f'/content/drive/My Drive/Quantum_chemistry/hardware_effient/{string_name}'
else:
    file_path = string_name

# Load the dictionary from the file
sparse_matrices_dict = pickle.load(open(file_path, 'rb'))

# Convert SciPy sparse COO matrices to PyTorch sparse COO tensors
sparse_matrices_dict = {key: scipy_to_torch_sparse(value) for key, value in sparse_matrices_dict.items()}
# Move PyTorch sparse COO tensors to GPU
sparse_matrices_dict = {key: value.to(device) for key, value in sparse_matrices_dict.items()}

In [ ]:
# In this case, it turns out we only use hopping only
for hopping_pair in sparse_matrices_dict:
    original_csr = sparse_matrices_dict[hopping_pair]

    # Transpose and convert back to CSR
    transposed_csr = original_csr.transpose(0, 1).to_sparse_csr()

    # Add the original and transposed matrices
    updated_csr = original_csr + transposed_csr

    # Replace the original matrix with the updated one
    sparse_matrices_dict[hopping_pair] = updated_csr


In [ ]:
##############################################################################
# 1) T-type operator (no chunking)
##############################################################################
def UCC_t_part_torch(psi, site_pair, theta):
    """
    T-type excitation:
        |psi> -> cos(theta)*|psi> + i*sin(theta)*(hop_matrix @ |psi>)
    applied for spin=+1 and spin=-1.
    """
    out = psi
    for spin in [1, -1]:
        key = site_pair + (spin,)
        # cos(...) mask
        cos_mask = torch.ones_like(out)
        idx = hopping_cose_dict[key]  # 1D index array
        cos_mask[idx] = torch.cos(theta).to(torch.complex128)
        part1 = cos_mask * out

        # hopping part
        hop_mat = sparse_matrices_dict[key]  # NxN sparse
        spmv = torch.sparse.mm(hop_mat, out.unsqueeze(1)).squeeze(1)
        part2 = (1j * torch.sin(theta).to(torch.complex128)) * spmv

        out = part1 + part2
    return out

##############################################################################
# 2) U-type operator (no chunking)
##############################################################################
def UCC_U_part_torch(psi, number_idx, theta):
    """
    U-type excitation:
       |psi> -> e^{ i * theta * n } |psi>
    where n = number_op_list[number_idx].
    """
    occ = number_op_list[number_idx]  # shape (N,) real
    phase = torch.exp(1j * theta * occ.to(torch.complex128))
    return psi * phase

##############################################################################
# 3) Apply all excitations in a single pass
##############################################################################
def apply_all_excitations(psi, x):
    """
    Applies every excitation in para_value_list (in order) to the wavefunction psi.
    Each entry in para_value_list is either:
      (u_index, param_idx) for U-type
      (siteA, siteB, param_idx) for T-type
    """
    out = psi
    for exc in para_value_list:
        if len(exc) == 2:
            # U-type
            out = UCC_U_part_torch(out, exc[0], x[exc[1]])
        else:
            # T-type
            out = UCC_t_part_torch(out, (exc[0], exc[1]), x[exc[2]])
    return out

##############################################################################
# 4) Classical optimizer: direct forward pass + energy
##############################################################################
def classical_optimizer(x):
    """
    1) Clone the global 'state_to_evolve'
    2) Apply all excitations once (no chunking/checkpointing)
    3) Compute energy with calculate_energy(h_sparse, final_psi)
    """
    psi_init = state_to_evolve.clone()
    psi_final = apply_all_excitations(psi_init, x)
    return calculate_energy(h_sparse, psi_final)

def fidelity_classical_optimizer(x):
    """
    1) Clone the global 'state_to_evolve'
    2) Apply all excitations once (no chunking/checkpointing)
    3) Compute energy with calculate_energy(h_sparse, final_psi)
    """
    psi_init = state_to_evolve.clone()
    psi_final = apply_all_excitations(psi_init, x)
    return calculate_fidelity(true_gs, psi_final)

In [ ]:
nn_condition = True

In [ ]:
u_index =[]
t_index =[]
def generate_parameters(para_repeat_times):
    global counting_parameter_index
    para_value_list = []
    counting_parameter_index = 0

    def can_commute(term, group):
        j, k, _ = term
        term_sites = {j, k}
        for existing_term in group:
            ej, ek, _ = existing_term
            existing_sites = {ej, ek}
            if term_sites & existing_sites:  # non-empty overlap => they don't commute
                return False
        return True

    # -------------------------------------------------------------
    # Main loop over para_repeat_times
    # -------------------------------------------------------------
    for iteration in range(para_repeat_times):
        # ---------------------------------------------------------
        # 1) Collect all hopping terms
        # ---------------------------------------------------------
        hopping_terms = []
        for j in range(N):
            for k in range(N):
                if nn_condition:
                    if j == k+1:
                        t_index.append(counting_parameter_index)
                        hopping_terms.append((j, k, counting_parameter_index))
                        counting_parameter_index += 1
                else:
                    if k < j:
                        t_index.append(counting_parameter_index)
                        hopping_terms.append((j, k, counting_parameter_index))
                        counting_parameter_index += 1
        # ---------------------------------------------------------
        # 2) Collect U-terms
        # ---------------------------------------------------------
        U_terms = []
        for i in range(N):
            u_index.append(counting_parameter_index)
            U_terms.append((i, counting_parameter_index))
            counting_parameter_index += 1

        # ---------------------------------------------------------
        # 3) Separate into NN vs. non-NN
        # ---------------------------------------------------------
        nn_terms = []
        non_nn_terms = []
        for t in hopping_terms:
            j, k, _ = t
            if j == k + 1:
                nn_terms.append(t)
            else:
                non_nn_terms.append(t)

        # ---------------------------------------------------------
        # 4) Build Group 1: (1,0) + commuting NN
        # ---------------------------------------------------------
        group1 = []
        leftover_nn_1 = []

        g1_special = None
        for t in nn_terms:
            if t[:2] == (1, 0):
                g1_special = t
                break

        if g1_special is not None:
            group1.append(g1_special)

        for t in nn_terms:
            if t == g1_special:
                continue
            if group1 and can_commute(t, group1):
                group1.append(t)
            else:
                leftover_nn_1.append(t)

        # ---------------------------------------------------------
        # 5) Build Group 2: (2,1) + commuting NN
        # ---------------------------------------------------------
        group2 = []
        leftover_nn_2 = []

        g2_special = None
        for t in leftover_nn_1:
            if t[:2] == (2, 1):
                g2_special = t
                break

        if g2_special is not None:
            group2.append(g2_special)

        for t in leftover_nn_1:
            if t == g2_special:
                continue
            if group2 and can_commute(t, group2):
                group2.append(t)
            else:
                leftover_nn_2.append(t)

        # leftover_nn_2 now has leftover NN terms
        # non_nn_terms are all non-NN from above
        leftover_terms = leftover_nn_2 + non_nn_terms

        # ---------------------------------------------------------
        # 6) Group leftover terms (normal commuting logic)
        # ---------------------------------------------------------
        leftover_commuting_groups = []
        for t in leftover_terms:
            placed = False
            for g in leftover_commuting_groups:
                if can_commute(t, g):
                    g.append(t)
                    placed = True
                    break
            if not placed:
                leftover_commuting_groups.append([t])

        # ---------------------------------------------------------
        # 7) Combine all groups, then shuffle them
        # ---------------------------------------------------------
        commuting_groups = []
        if group1:
            commuting_groups.append(group1)
        if group2:
            commuting_groups.append(group2)
        commuting_groups.extend(leftover_commuting_groups)

        # # Shuffle all groups for this iteration
        # random.shuffle(commuting_groups)

        # ---------------------------------------------------------
        # 8) Flatten them and append U-terms
        # ---------------------------------------------------------
        ordered_hopping_list = [term for group in commuting_groups for term in group]
        para_value_list.extend(ordered_hopping_list)
        para_value_list.extend(U_terms)

        #####(Optional) Print for debugging
        # print(f"\nIteration {iteration} / {para_repeat_times}, commuting groups (randomly ordered):")
        # for idx, group in enumerate(commuting_groups):
        #     print(f"  Group {idx+1}: {group}")

    return para_value_list


In [ ]:
para_repeat_times=40
para_value_list = generate_parameters(para_repeat_times)
print(len(para_value_list))

920


In [ ]:
# input value to check energy
x = np.load('predicted_limit_epsilon.npy')
x = torch.tensor(x, dtype=torch.float64, device=device, requires_grad=False)
classical_optimizer(x)

tensor(-19.3743047250, device='cuda:0', dtype=torch.float64)

In [ ]:
#-19.3648826728 , n=10

tensor(-19.4132131926, device='cuda:0', dtype=torch.float64)

In [ ]:
if IN_COLAB:
    filename = f'/content/drive/My Drive/Quantum_chemistry/result/log_{mole_name}_bond_{bond_length}_para_{para_repeat_times}.txt'
    checkpoint_filename = f'/content/drive/My Drive/Quantum_chemistry/result/checkpoint_{mole_name}_bond_{bond_length}_para_{para_repeat_times}.pt'
else:
    filename = f'log_{mole_name}_bond_{bond_length}_para_{para_repeat_times}.txt'
    checkpoint_filename = f'checkpoint_{mole_name}_bond_{bond_length}_para_{para_repeat_times}.pt'
lr = 0.1
energy_threshold = 1e-8 # Stopping condition for cost function change
gradient_norm_threshold =1e-4
max_epochs = 5000
write_file_condition = True
# Initialize x if checkpoint does not exist
if os.path.isfile(checkpoint_filename):
    checkpoint = torch.load(checkpoint_filename,weights_only=False)
    x = torch.tensor(checkpoint['x'], dtype=torch.float64, device=device, requires_grad=True)
    # Initialize L-BFGS optimizer
    optimizer = optim.LBFGS([x], lr=lr)
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
else:
    initial_guess = np.pi/3*np.ones( counting_parameter_index)
    #initial_guess = np.random.uniform(0,2*3.14159, counting_parameter_index)
    # initial_guess = np.zeros(counting_parameter_index)
    # initial_guess[t_index] = np.ones(len(t_index))*(-np.pi/64)
    # initial_guess[t_index] = np.random.uniform(-np.pi/128,np.pi/128, len(t_index))
    x = torch.tensor(initial_guess, dtype=torch.float64, device=device, requires_grad=True)
    optimizer = optim.LBFGS([x], lr=lr, max_iter=20, history_size=100,line_search_fn='strong_wolfe')
    epoch = 0
prev_energy = None  # Store previous energy for stopping condition
def closure():
    optimizer.zero_grad()
    energy = classical_optimizer(x)
    energy.backward()
    return energy
with open(filename, 'a') as f:  # Use 'a' to append to the log file if it exists
    while epoch < max_epochs:
        optimizer.step(closure)
        # Compute gradient norm
        gradient_norm = x.grad.norm().cpu().detach().numpy()
        # Compute current energy
        current_energy = classical_optimizer(x).cpu().detach().numpy()
        # Compute energy difference if previous energy exists
        energy_diff = abs(current_energy - prev_energy) if prev_energy is not None else float('inf')
        prev_energy = current_energy  # Update previous energy
        result_str = f"Epoch {epoch + 1}, Energy: {current_energy:.6f}, Gradient Norm: {gradient_norm:.6f}, Energy Diff: {energy_diff:.8f}\n"
        print(result_str, end='')
        if write_file_condition:
            # Log and save data
            if (epoch + 1) % 1 == 0:
                f.write(result_str)
                # Append latest x values to the end of the file
                x_values = ','.join([str(val.item()) for val in x.detach().cpu().numpy()])
                f.write(f"Latest x values at epoch {epoch + 1}: {x_values}\n")
                # Save checkpoint
                torch.save({
                    'epoch': epoch,
                    'x': x.detach().cpu().numpy(),  # Save as numpy array for checkpoint
                    'optimizer_state_dict': optimizer.state_dict(),
                }, checkpoint_filename)

        # **Stopping condition: both gradient norm and energy difference must be small**
        if  energy_diff < energy_threshold or gradient_norm < gradient_norm_threshold :
            print(f"Stopping at epoch {epoch + 1} due to small energy difference.")
            break
        epoch += 1
        f.flush()

In [ ]:
fidelity_classical_optimizer(x)

In [ ]:
# # test spin
# # Load the sparse matrix in pytorch COO format with complex 128 data
# total_S = scipy.sparse.load_npz('total_S_norb=10_na_7_nb_7.npz').tocoo()
# total_S = scipy_to_torch_sparse(total_S)
# total_S = total_S.to(device)

In [ ]:
filename = f'final_report_{mole_name}_bond_{bond_length}_para_{para_repeat_times}.txt'

In [ ]:
# once done write result file

#outcome result
length = len(para_value_list)

# Determine if tensor 'x' is on GPU or CPU
if x.is_cuda:
    # GPU version
    final_energy_diff = (classical_optimizer(x).detach().cpu() - true_gs_energy).cpu().numpy()
else:

    # CPU version
    final_energy_diff = (classical_optimizer(x).detach() - true_gs_energy).numpy()

######################################################################

final_fidelity = fidelity_classical_optimizer(x)
x_values = ','.join([str(val.item()) for val in x.detach().cpu().numpy()])
# Write results to file
with open(filename, 'w') as file:
    file.write(f"Final Energy Diff: {final_energy_diff}\n")
    file.write(f"Final Fidelity: {final_fidelity}\n")
    file.write(f"Optimized X Values: {x_values}\n")
    file.write(f"Length of parameter value list: {length}\n")
    file.write(f"Epoch : {epoch }\n")